In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
%sql
drop catalog if exists goldscm cascade;
create catalog goldscm;

In [0]:
%sql
USE catalog goldscm;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS goldscm.gold_schema;

**LOAD THE SILVER DATA**

In [0]:
%python
# Load Customer Data
df_cust = spark.read.format("delta") \
 .option("header", "true") \
 .option("inferSchema", "true") \
 .load('abfss://silver@scmdataset2025.dfs.core.windows.net/Customer')
# Load Orders Data
df_orders = spark.read.format("delta") \
 .option("header", "true") \
 .option("inferSchema", "true") \
 .load('abfss://silver@scmdataset2025.dfs.core.windows.net/Orders')
# Load Warehouse Data
df_warehouse = spark.read.format("delta") \
 .option("header", "true") \
 .option("inferSchema", "true") \
 .load('abfss://silver@scmdataset2025.dfs.core.windows.net/Warehouse')
# Load Logistics Data
df_logistics = spark.read.format("delta") \
 .option("header", "true") \
 .option("inferSchema", "true") \
 .load('abfss://silver@scmdataset2025.dfs.core.windows.net/Logistics')
# Load Products Data
df_products = spark.read.format("delta") \
 .option("header", "true") \
 .option("inferSchema", "true") \
 .load('abfss://silver@scmdataset2025.dfs.core.windows.net/Product')
# Load Supply Data
df_supply = spark.read.format("delta") \
 .option("header", "true") \
 .option("inferSchema", "true") \
 .load('abfss://silver@scmdataset2025.dfs.core.windows.net/Supply')

In [0]:
df_orders.display(5)

**Orders vs Total Cost and Shipment Cost**

In [0]:
print(df_orders.columns)
print(df_logistics.columns)

In [0]:
%python
# Ensure that both DataFrames have the 'Order_ID' column
df_orders = df_orders.withColumnRenamed("Order_ID", "Order_ID")
df_logistics = df_logistics.withColumnRenamed("Order_ID", "Order_ID")

# Perform the join operation
df_order_shipment_cost = df_orders.join(
    df_logistics,
    df_orders["Order_ID"] == df_logistics["Order_ID"]
).select(
    df_orders["Order_ID"],
    df_orders["Total_Cost"],
    df_logistics["Shipment_Cost"]
).orderBy("Order_ID", ascending=True)

# Display the result
display(df_order_shipment_cost)

In [0]:
df_order_shipment_cost = df_orders.join(df_logistics, "Order_ID") \
 .select("Order_ID", "Total_Cost", "Shipment_Cost") \
 .orderBy("Order_ID", ascending=True)
df_order_shipment_cost.display()

**Order_Date Vs Shipment_Date Vs Delivery_Date**

In [0]:
df_orders_delivery_info = df_orders.join(df_logistics, "Order_ID") \
 .select(df_orders.Order_ID, df_orders.Order_Date,
df_logistics.Shipment_Date, df_logistics.Delivery_Date) \
 .orderBy("Order_ID")
df_orders_delivery_info.display()

**Total Orders Revenue Over Time**

In [0]:
orders_summary = df_orders.groupBy("Order_Date") \
 .agg(sum("Total_Cost").alias("Total_Revenue")) \
 .orderBy("Order_Date", ascending=True)
display(orders_summary)

**Top-Selling Products**

In [0]:
from pyspark.sql.functions import sum, col, round

total_quantity = df_orders.agg(sum("Quantity_Ordered").alias("Total_Quantity")).collect()[0]["Total_Quantity"]
top_selling_quantities = df_orders.join(df_products, "Product_ID") \
 .groupBy("Product_ID", "Product_Name") \
 .agg(sum("Quantity_Ordered").alias("Total_Quantity_Ordered")) \
 .withColumn("Percentage_of_Allotment",
round((col("Total_Quantity_Ordered") / total_quantity) * 100, 2)) \
 .orderBy("Total_Quantity_Ordered", ascending=False)
top_selling_quantities.display()

**Most Orders By Customer**

In [0]:
most_orders_by_customer = df_orders.join(df_cust,
"Customer_ID").groupBy("Customer_ID", "Customer_Location").agg(
 count("Order_ID").alias("Total_orders")
).orderBy("Total_orders", ascending=False)
most_orders_by_customer.display()


**Warehouse wise Top-Orders**

In [0]:
warehouse_wise_top_orders = df_orders.join(df_warehouse,
"Warehouse_ID") \
 .groupBy("Warehouse_ID", "Warehouse_Location") \
 .agg(count("Order_ID").alias("Total_Orders")) \
 .orderBy("Total_Orders", ascending=False)
warehouse_wise_top_orders.display()

**Customer Location vs Order Volume**

In [0]:
%python
Cust_behavior = df_cust.join(df_orders, "Customer_ID") \
    .join(df_products, "Product_ID") \
    .groupBy("Customer_Location", "Product_Name") \
    .agg(sum("Quantity_Ordered").alias("Total_Quantity_Ordered")) \
    .orderBy("Total_Quantity_Ordered", ascending=False)

display(Cust_behavior)

**Customer Order Frequency**

In [0]:
from pyspark.sql.functions import countDistinct
df_customer_order_frequency = df_orders.join(df_cust,
"Customer_ID").groupBy(
 "Customer_ID", "Customer_Location"
).agg(
 countDistinct("Order_Date").alias("Order_Frequency")
).orderBy( "Order_Frequency", ascending=False)

df_customer_order_frequency.display()

**Average delivery time by Warehouse**

In [0]:
average_delivery_time = df_logistics.withColumn(
 "Delivery_Time",
 datediff(col("Delivery_Date"), col("Shipment_Date"))
).groupBy(
 "From", "To"
).agg(
 round(avg("Delivery_Time"), 2).alias("Average_Delivery_Time")
).orderBy("From")
average_delivery_time.display()

**Shipping Cost by Distance**

In [0]:
# Define Distance Ranges
shipping_cost_by_distance = df_logistics.withColumn(
 "Distance_Range",
 when(col("Distance") <= 50, "0-50 km")
 .when((col("Distance") > 50) & (col("Distance") <= 100), "51-100km")
 .when((col("Distance") > 100) & (col("Distance") <= 200), "101-200km")
 .when((col("Distance") > 200) & (col("Distance") <= 500), "201-500km")
 .otherwise("500+ km")).groupBy("Distance_Range") \
 .agg(sum("Shipment_Cost").alias("Total_Shipping_Cost")) \
 .orderBy("Distance_Range")
shipping_cost_by_distance.display()

**Supplier Wise Product Distribution**


In [0]:
df_supply_wise_pro=df_supply.join(df_products,"Product_ID")\
 .groupBy("Supplier_ID","Product_Name")\
 .agg(sum("Manufacturing_Unit").alias("Total_Manufacturing_Units"))\
 .orderBy("Supplier_ID",ascending=True)
df_supply_wise_pro.display()

**Manufacturing units by Location**

In [0]:
df_manufacturing_unit=df_supply.groupBy("Supplier_City")\
 .agg(sum("Manufacturing_Unit").alias("Total_Manufacturing_Units"))\
 .orderBy("Total_Manufacturing_Units",ascending=False)
df_manufacturing_unit.display()

In [0]:
df_logistics=df_logistics.withColumnRenamed("From","Warehouse_Location")
df_logistics=df_logistics.withColumnRenamed("To","Customer_Location")
df_orders=df_orders.withColumnRenamed("Total_Cost","Order_Cost")


In [0]:
f_joinedtable = df_logistics.join(df_orders, "Order_id", "inner") \
 .join(df_cust, "Customer_ID", "inner") \
 .join(df_products, "Product_ID", "inner")\
 .select(df_orders.Order_ID,
 df_products.Product_ID,
 df_cust.Customer_ID,
 df_orders.Warehouse_ID,
 df_products.Product_Name,
 df_products.Category,
 df_orders.Quantity_Ordered,
 df_logistics.Warehouse_Location,
 df_cust.Customer_Location,
 df_logistics.Distance,
 df_orders.Order_Cost,
 df_orders.Total_COGs,
 df_logistics.Shipment_Cost,
 df_logistics.Order_Date,
 df_logistics.Shipment_Date,
 df_logistics.Delivery_Date
 ).distinct().orderBy(df_orders.Order_ID)
f_joinedtable.display()

In [0]:
f_joinedtable.display()

In [0]:
f_joinedtable.write\
    .format("delta")\
    .mode("overwrite")\
    .option("inferSchema", "true")\
    .saveAsTable("goldscm.gold_schema.f_joined_table")

In [0]:
f_joinedtable.write.format("delta")\
    .mode("overwrite")\
    .option("inferSchema", "true")\
    .save("abfss://gold@scmdataset2025.dfs.core.windows.net/f_joined_table")

In [0]:
f_joinedtable.printSchema()

In [0]:
df_warehouse.write\
    .format("delta")\
    .mode("overwrite")\
    .option("inferSchema", "true")\
    .saveAsTable("goldscm.gold_schema.Warehouse_table")

In [0]:
df_warehouse.write.format("delta")\
    .mode("overwrite")\
    .option("inferSchema", "true")\
    .save("abfss://gold@scmdataset2025.dfs.core.windows.net/df_warehouse")

In [0]:
df_supply.write \
    .format("delta") \
    .mode("overwrite") \
    .option("inferSchema", "true") \
    .saveAsTable("goldscm.gold_schema.Supplier_table")

In [0]:
df_supply.write.format("delta")\
    .mode("overwrite")\
    .option("inferSchema", "true")\
    .save("abfss://gold@scmdataset2025.dfs.core.windows.net/Supplier_table")